In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib qt
#%matplotlib widget

In [ ]:
import os
import sys
from pathlib import Path

src_dir = str(Path(os.getcwd()).parent / "src")
sys.path.insert(1, src_dir)
#os.environ["PYTHONPATH"] = src_dir

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import Normalize, ListedColormap
import config
from utils import set_size, print_header
import scienceplots
import pickle
import data_loader
import pandas
from mcs_function import SimulationAttenuation
from mpl_toolkits.axes_grid1 import AxesGrid, ImageGrid
from PIL import Image

plt.style.use('science')

In [ ]:
ground_truth_color = "orange"
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [ ]:
colors

In [ ]:
molecule_names = ["Hb02", "Hbb", "oxCCO", "redCOO", "H20", "Fat"]

In [ ]:
molecules = ["Hb02", "Hbb", "oxCCO", "redCOO", "H20", "Fat"]

In [ ]:
molecules_units = [r"$c_\text{Hb02}$ [$\mu$M]", r"$c_\text{Hbb}$ [$\mu$M]", r"$c_\text{oxCCO}$ [$\mu$M]", r"$c_\text{redCCO}$ [$\mu$M]", r"$f_\text{H20}$ [\%]", r"$f_\text{Fat}$ [\%]"]

In [ ]:
doc_width = 516.0

# Background

### Absorption spectra

In [ ]:
wavelengths=np.linspace(520, 900, 5000)
mu_a_matrix = data_loader.DataLoader.absorption_coefs(
    wavelengths,
    use_diff_oxycco=False,
    use_water_and_fat=True
)

In [ ]:
fig, ax1 = plt.subplots(1, 1, figsize=set_size(doc_width, subplots=(1, 1)), layout='constrained')
handles = []
ax1.plot(wavelengths, mu_a_matrix[:, :4], label=molecule_names[:4])
ax1.set_xlabel="Wavelengths (nm)"
ax1.set_ylabel("Molar absorption coefficient [cm$^{-1}$mM$^{-1}$]")
ax2 = ax1.twinx()
ax2.plot(wavelengths, mu_a_matrix[:, -2], color=colors[4], label=molecule_names[4])
ax2.plot(wavelengths, mu_a_matrix[:, -1], color=colors[5], label=molecule_names[5])
ax2.set_ylim(0, 0.1)
ax2.set_ylabel("Absorption coefficient [cm$^{-1}$]")
fig.legend(loc="upper right", ncols=2, bbox_to_anchor=(0.9, 0.95))
fig.tight_layout()

fig.savefig(config.eval_dir/"chromophore_absorptions.pdf")

In [ ]:
fig, ax3 = plt.subplots(1, 1, figsize=set_size(doc_width, subplots=(1, 1)), layout='constrained')
for tissue in ["gray matter", "blood vessel", "tumor"]:
    a, b = data_loader.DataLoader.tissue_parameters[tissue][1]
    g = data_loader.DataLoader.tissue_parameters[tissue][2]
    mu_s_red = a * np.power(wavelengths / 500, -b) / (1-g)
    ax3.plot(
        wavelengths,
        mu_s_red,
        label=tissue.capitalize
    )
fig.savefig(config.eval_dir/"scattering.pdf")

In [ ]:
fig, (ax1, ax3) = plt.subplots(2, 1, figsize=set_size(doc_width, subplots=(2, 1)), layout='constrained')
handles = []
ax1.plot(wavelengths, mu_a_matrix[:, :4], label=molecules[:4])
ax1.set_xlabel="Wavelengths (nm)"
ax1.set_ylabel("Absorption coefficient $(cm^{-1}mM^{-1}$)")
ax2 = ax1.twinx()
ax2.plot(wavelengths, mu_a_matrix[:, -2], color=colors[4], label=molecules[4])
ax2.plot(wavelengths, mu_a_matrix[:, -1], color=colors[5], label=molecules[5])
ax2.set_ylim(0, 0.1)
ax2.set_ylabel("Absorption $(cm^{-1})$")
ax2.legend(loc="upper right", ncols=2, bbox_to_anchor=(0.9, 0.95))

for tissue in ["gray matter", "blood vessel", "tumor"]:
    a, b = data_loader.DataLoader.tissue_parameters[tissue][1]
    g = data_loader.DataLoader.tissue_parameters[tissue][2]
    mu_s_red = a * np.power(wavelengths / 500, -b) / (1-g)
    ax3.plot(
        wavelengths,
        mu_s_red
    )

#fig.tight_layout()

fig.savefig(config.eval_dir/"absorption_and_scattering.pdf")

# Evaluation

## Simulated

In [ ]:
with open(config.simulated_dataset_dir / "simulation_data.pickle", "rb") as f:
    wavelengths, simulation_data = pickle.load(f)

In [ ]:
mu_a_matrix = data_loader.DataLoader.absorption_coefs(
    wavelengths,
    use_diff_oxycco=False,
    use_water_and_fat=True
)

### Table of baseline concentrations

In [ ]:
baseline_table = np.empty((10, 6), dtype=object)
molecule_param_names = [r"$c_\text{Hb02}$ [$\mu$M]", r"$c_\text{Hbb}$ [$\mu$M]", r"$c_\text{oxCCO}$ [$\mu$M]", r"$c_\text{redCCO}$ [$\mu$M]", r"$f_\text{H20}$ [\%]", r"$f_\text{Fat}$ [\%]"]
baseline_table[:6, 0] = molecule_param_names
baseline_table[-4:, 0] = [r"a [cm$^{-1}$]", "b", "g", "n"]

for tissue_idx, tissue in enumerate(list(simulation_data.keys())):
    baseline_table[:-4, tissue_idx+1] = simulation_data[tissue][0][:, 0]
    baseline_table[-4:-2, tissue_idx+1] = simulation_data[tissue][1][:, 0]
    baseline_table[-2, tissue_idx+1] = data_loader.DataLoader.tissue_parameters[tissue][2]
    baseline_table[-1, tissue_idx+1] = data_loader.DataLoader.tissue_parameters[tissue][3]


baseline_table[[0, 1, 4, 5, 7], -1] = 0.15
baseline_table[[2, 3], -1] = 5e-4
baseline_table[6, -1] = 5
baseline_table[-2:, -1] = 0

# concentrations are in mM, volume fractions in dimensionless (i.e. not yet in percent)
# -> multiply by concentrations by thousand to obtain uM and volume fractions by 100 to obtain 
baseline_table[:4, 1:] *= 1000
baseline_table[4:6, 1:] *= 100

np.savetxt(
    config.eval_dir/"simulated/baseline_table.txt",
    baseline_table, fmt="%s",
    comments="",
    delimiter="\t",
    header="\t".join(map(str.capitalize, ["parameter"] + list(simulation_data.keys()) + ["sigma"]))
)

### Absorption and scattering spectra of simulated tissue

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=set_size(width=doc_width, subplots=(1, 2)))
handles = []
for tissue in list(simulation_data.keys()):
    mu_a = mu_a_matrix @ simulation_data[tissue][0][:, 0]
    a, b = simulation_data[tissue][1][:, 0]
    mu_s_red = a * np.power(wavelengths / 500, -b)
    handles.append(
        ax[0].plot(
            wavelengths,
            mu_a,
            label=tissue
        )[0]
    )

    ax[1].plot(
        wavelengths,
        mu_s_red,
        label=tissue
    )

ax[0].legend()
ax[1].legend()




### Absorbance of simulated tissue

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=set_size(width=doc_width))
for tissue in list(simulation_data.keys()):
    ax.plot(
        wavelengths,
        simulation_data[tissue][-1][:, 0],
        label=tissue.capitalize()
    )

#ax.set_title("Absorbance spectra from literature values")
ax.set(xlabel='Wavelength [nm]', ylabel='Absorbance')
ax.legend()

fig.savefig(config.eval_dir/"simulated/simulated_absorbance.pdf")

### MCS, Diffusion, Jacques PL comparison

In [ ]:
print_header(config.eval_dir / "simulated/forward/pl_comp_gray matter.dat")

In [ ]:
pl_comp_data_gray_matter = np.loadtxt(config.eval_dir/"simulated/forward/pl_comp_gray matter.dat", skiprows=1)
pl_comp_data_blood_vessel = np.loadtxt(config.eval_dir/"simulated/forward/pl_comp_blood vessel.dat", skiprows=1)
sl_comp_data_gray_matter = np.loadtxt(config.eval_dir/"simulated/forward/sl_comp_gray matter.dat", skiprows=1)
sl_comp_data_blood_vessel = np.loadtxt(config.eval_dir/"simulated/forward/sl_comp_blood vessel.dat", skiprows=1)
pl_comp_data_tumor = np.loadtxt(config.eval_dir/"simulated/forward/pl_comp_tumor.dat", skiprows=1)
sl_comp_data_tumor = np.loadtxt(config.eval_dir/"simulated/forward/sl_comp_tumor.dat", skiprows=1)

In [ ]:
fig, ax = plt.subplots(3, 2, figsize=set_size(width=doc_width, subplots=(3, 2)), layout='constrained')
cur_colors = [colors[5], colors[3]]


for row, (pl_comp_data, sl_comp_data) in enumerate([(pl_comp_data_gray_matter, sl_comp_data_gray_matter), (pl_comp_data_blood_vessel, sl_comp_data_blood_vessel), (pl_comp_data_tumor, sl_comp_data_tumor)]):
    ax[row, 0].plot(
        pl_comp_data[:, 0],
        pl_comp_data[:, 1],
        color=ground_truth_color
    )
    
    ax[row, 1].plot(
        sl_comp_data[:, 0],
        sl_comp_data[:, 1],
        color=ground_truth_color
    )

    for i in [1, 0]:
        ax[row, 0].plot(
            pl_comp_data[:, 0],
            pl_comp_data[:, i+2],
            color=cur_colors[i]
        )

        ax[row, 1].plot(
            sl_comp_data[:, 0],
            sl_comp_data[:, i+2],
            color=cur_colors[i]
        )

    ax[row, 0].set(xlabel="Wavelengths [nm]", ylabel=r"$\bar{L}_p$ [cm]")
    ax[row, 1].set(xlabel="Wavelengths [nm]", ylabel=r"$\bar{L}_s$ [cm]")

ax[0, 0].set_title(r"$\bar{L}_p$, gray matter tissue")
ax[0, 1].set_title(r"$\bar{L}_s$, gray matter tissue")
ax[1, 0].set_title(r"$\bar{L}_p$, blood vessel tissue")
ax[1, 1].set_title(r"$\bar{L}_s$, blood vessel tissue")
ax[2, 0].set_title(r"$\bar{L}_p$, tumor tissue")
ax[2, 1].set_title(r"$\bar{L}_s$, tumor tissue")

handles, _ = ax[0, 0].get_legend_handles_labels()
fig.legend(
    handles,
    labels=["MC simulation", "Jacques, tissue-specific", "$\delta$-P1"],
    ncols=3,
    loc="outside lower center"
)
fig.savefig(config.eval_dir / "simulated/forward/pl_sl_comparison.pdf")

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=set_size(width=doc_width, subplots=(1, 2)), layout='constrained')
cur_colors = [colors[5], colors[3]]

ax[0].plot(
    pl_comp_data_tumor[:, 0],
    pl_comp_data_tumor[:, 1],
    color=ground_truth_color
)

ax[1].plot(
    sl_comp_data_tumor[:, 0],
    sl_comp_data_tumor[:, 1],
    color=ground_truth_color
)
for i in [1, 0]:
    ax[0].plot(
        pl_comp_data_tumor[:, 0],
        pl_comp_data_tumor[:, i+2],
        color=cur_colors[i]
    )
    ax[1].plot(
        sl_comp_data_tumor[:, 0],
        sl_comp_data_tumor[:, i+2],
        color=cur_colors[i]
    )
ax[0].set(xlabel="Wavelengths (nm)", ylabel=r"Pathlength/$D_a$ (cm)")
ax[1].set(xlabel="Wavelengths (nm)", ylabel=r"$D_s$(cm)")

ax[0].set_title("$D_a$ comparison, tumor tissue")
ax[1].set_title("$D_s$ comparison, tumor tissue")

handles, _ = ax[0].get_legend_handles_labels()
fig.legend(
    handles,
    labels=["Ground truth", "Jacques, specific", "Diffusion"],
    ncols=3,
    loc="outside lower center"
)
fig.savefig(config.eval_dir / "simulated/forward/tumor_pl_sl_comparison.pdf")

### Jacques-m-param analysis

In [ ]:
with open(config.m_params_path, "rb") as f:
    m_params, A_vals, N_vals, dref_vals = pickle.load(f)

In [ ]:
def A_j_jacques(N_vals, m1, m2, m3):
    m1 + m2*np.exp(np.log(N_vals)/m3)

In [ ]:
fig, ax = plt.subplots(4, 1, figsize=set_size(width=doc_width, height=7.4, subplots=(4, 1)))
colors = ["black", "darkgrey", "darkred", "darkblue"]
assert np.allclose(np.array([len(N_vals[tissue]) for tissue in ["gray matter", "blood vessel"]])-len(N_vals["tumor"]), 0)
num_vals_tissue = len(N_vals['gray matter'])

for i, tissue in enumerate([ "general", "gray matter", "blood vessel", "tumor"]):
    N_vals_interp = np.linspace(min(N_vals[tissue]), max(N_vals[tissue]), 100)
    m1, m2, m3 = m_params[tissue]
    # move each axis slightly upwards
    box = ax[i].get_position()
    box.y0 -= 0.03
    box.y1 -= 0.03
    ax[i].set_position(box)
    ax[i].set_xscale("log")
    ax[i].set_title(f"{tissue.capitalize() + ' tissue' if tissue != 'general' else 'Tissue-independent'}")
    ax[i].text(
        0.425, 0.6,
        f"$m_1$: {m1:.3}\n$m_2$: {m2:.3}\n$m_3$: {m3:.3}",
        transform=ax[i].transAxes
    )
    label = "$A_j$ using fitted m-parameters" if tissue=="general" else None
    res = ax[i].plot(
        N_vals_interp,
        m1 + m2*np.exp(np.log(N_vals_interp)/m3),
        color=colors[i],
        label=label
    )
    ax[i].set(xlabel="$N = \mu_s'/\mu_a$", ylabel="$A_j$")
    if tissue == "general":
        lower_idx = 0
        colors2 = ["darkgray", "darkred", "darkblue", "black"]
        for j in range(4):
            label = "Sampled tissue values" if j==3 else None
            res = ax[i].scatter(
                N_vals[tissue][lower_idx:lower_idx+num_vals_tissue],
                A_vals[tissue][lower_idx:lower_idx+num_vals_tissue],
                color=colors2[j],
                label=label
            )
        
            lower_idx += num_vals_tissue
    else:
        ax[i].scatter(
            N_vals[tissue],
            A_vals[tissue],
            color=colors[i]
        )

fig.legend(
    ncols=2,
    loc="outside lower center",
    bbox_to_anchor=[0.5, 0]
)
plt.tight_layout()

fig.savefig(config.eval_dir/"simulated/forward/jacques_params.pdf")


### Forward

In [ ]:
print_header(config.eval_dir / "simulated/forward/gray matter.dat")

In [ ]:
data_gray_matter = np.loadtxt(config.eval_dir / "simulated/forward/gray matter.dat", skiprows=1)
data_artery = np.loadtxt(config.eval_dir / "simulated/forward/artery.dat", skiprows=1)
data_vein = np.loadtxt(config.eval_dir / "simulated/forward/vein.dat", skiprows=1)
data_tumor = np.loadtxt(config.eval_dir / "simulated/forward/tumor.dat", skiprows=1)

In [ ]:
fig, ax = plt.subplots(4, 2, figsize=set_size(width=doc_width, subplots=(4, 2)), layout='constrained')
pparam = dict(xlabel="Wavelength [nm]", ylabel="Absorbance")
lines = np.empty((4, 7), dtype=object)

for row, data in enumerate([data_gray_matter, data_artery, data_vein, data_tumor]):
    # plot ground truth in all subplots
    for col in range(2):
        lines[row, 0] = ax[row, col].plot(
            data[:, 0],
            data[:, 1],
            color=ground_truth_color,
        )[0]
            
    for i, line_idx in enumerate([2, 3]):
        lines[row, i+1] = ax[row, 0].plot(
            data[:, 0],
            data[:, line_idx],
            color=colors[i]
        )[0]
    
    for i, line_idx in enumerate([6, 7, 8, 9]):
        lines[row, i+3] = ax[row, 1].plot(
            data[:, 0],
            data[:, line_idx],
            color=colors[i+2]
        )[0]

for axis in ax.flat:
    axis.set(**pparam)

ax[0, 0].set_title("Linear models, gray matter tissue")
ax[0, 1].set_title("Nonlinear models, gray matter tissue")
ax[1, 0].set_title("Linear models, artery tissue")
ax[1, 1].set_title("Nonlinear models, artery tissue")
ax[2, 0].set_title("Linear models, vein tissue")
ax[2, 1].set_title("Nonlinear models, vein tissue")
ax[3, 0].set_title("Linear models, tumor tissue")
ax[3, 1].set_title("Nonlinear models, tumor tissue")

#fig.subplots_adjust(hspace=0.3, wspace=0.3)
fig.legend(
    handles=list(lines[0, :]), 
    labels=[
        "MC simulation", "MBLL+MC", "MBLL+$\delta$-P1", "Jacques, specific", "Jacques, general",
        "$\delta$-P1", "Inverse MC"
    ],
    loc="outside lower center",
    ncols=4,
    #bbox_to_anchor=(1,-0.1),
    #bbox_transform=fig.transFigure
)
fig.savefig(config.eval_dir / "simulated/forward/simulated_forward.pdf")

Plots for MCS const, MCS const GM PL

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=set_size(width=doc_width, subplots=(2, 2)), layout='constrained')
pparam = dict(xlabel="Wavelength [nm]", ylabel="Absorbance")
lines = np.empty((4, 3), dtype=object)

for idx, (ax, data) in enumerate(zip(axs.flat, [data_gray_matter, data_artery, data_vein, data_tumor])):
    lines[idx, 0] = ax.plot(
        data[:, 0],
        data[:, 1],
        color=ground_truth_color
    )[0]

    lines[idx, 1] = ax.plot(
        data[:, 0],
        data[:, 4],
        color=colors[0]
    )[0]

    lines[idx, 2] = ax.plot(
        data[:, 0],
        data[:, 5],
        color=colors[1]
    )[0]

    ax.set(**pparam)
    ax.set_title(f"{['gray matter', 'artery', 'vein', 'tumor'][idx].capitalize()} tissue")

#fig.subplots_adjust(hspace=0.3, wspace=0.3)
fig.legend(
    handles=list(lines[0, :]), 
    labels=[
        "MC simulation", r"MBLL, $\bar{L}_p=1$ and $\bar{L}_s=0$", r"MBLL, gray matter $\bar{L}_p$ and $\bar{L}_s$"
    ],
    loc="outside lower center", 
    ncols=3,
    #bbox_to_anchor=(1,-0.1),
    #bbox_transform=fig.transFigure
)
fig.savefig(config.eval_dir / "simulated/forward/simulated_forward_mbll_const_pls.pdf")

### Backward

#### Bar plot

In [ ]:
with open(config.eval_dir/"simulated/backward/model_indices.txt", "r") as f:
    print(f.read())

In [ ]:
with open(config.eval_dir/"simulated/backward/bar_plot_data.pickle", "rb") as f:
    bar_plot_data = pickle.load(f)

In [ ]:
bar_plot_data.shape

Normalize bar-plot-data by GT

In [ ]:
bar_plot_data_normed = bar_plot_data[:, :, :] / bar_plot_data[:, [0], :]

In [ ]:
np.unravel_index(np.argmax(bar_plot_data_normed), bar_plot_data.shape)

In [ ]:
_, num_models, num_molecules = bar_plot_data_normed.shape

In [ ]:
models = [
    "Ground truth", "MBLL+MC", "MBLL+$\delta$-P1", r"MBLL, $\bar{L}_p=1$", r"MBLL, gray matter $\bar{L}_p$", "Jacques, specific", "Jacques, general", "$\delta$-P1",
    "Inverse MC", "Inverse MC, sparse"
]

In [ ]:
len(models)

In [ ]:
#fig, ax = plt.subplots(4, 1, figsize=set_size(width=doc_width, subplots=(4, 1)), layout='constrained')
#x = np.arange(bar_plot_data.shape[1])
#width = 0.125
#
#for tissue_idx, tissue in enumerate(simulation_data.keys()):
#    for molecule_idx, molecule_str in enumerate(molecules):
#        offset = width * molecule_idx
#        rects = ax[tissue_idx].bar(x + offset, bar_plot_data[tissue_idx, :, molecule_idx], width, label=molecule_str)
#        #ax[tissue_idx].bar_label(rects, padding=3)
#
#
#    ax[tissue_idx].set(ylabel="Concentration Percentage to ground truth")
#    ax[tissue_idx].set_title("Reconstructed concentrations for simulated dataset")
#    ax[tissue_idx].set_xticks(x + width, models)
#    ax[tissue_idx].legend(loc="upper left", ncols=1)
#    ax[tissue_idx].set_ylim(0, 10)

In [ ]:
y = np.arange(bar_plot_data_normed.shape[1])
width = 0.125

for fig_idx in range(2):
    fig, ax = plt.subplots(2, 1, figsize=set_size(width=doc_width, subplots=(2, 1)), layout='constrained')

    for tissue_idx, tissue in enumerate(["gray matter", "artery", "vein", "tumor"][2*fig_idx:2*fig_idx+2], start=2*fig_idx):
        ax_idx = tissue_idx - fig_idx*2
        for molecule_idx, molecule_str in enumerate(molecules):
            offset = width * molecule_idx
            rects = ax[ax_idx].barh(y + offset, bar_plot_data_normed[tissue_idx, :, molecule_idx], width, label=molecule_str)
            #ax[tissue_idx].bar_label(rects, padding=3)


        ax[ax_idx].set(xlabel="Concentration fraction to ground truth")
        #ax[ax_idx].set_title(f"Reconstructed concentrations, {tissue} tissue")
        ax[ax_idx].set_title(f"{tissue.capitalize()} tissue")
        ax[ax_idx].set_yticks(y + width, models)
        ax[ax_idx].invert_yaxis()
        ax[ax_idx].legend(loc="lower left", ncols=1)
        ax[ax_idx].set_xlim(-2, 6)

    fig.savefig(config.eval_dir/f"simulated/backward/concentrations_bar_plot{fig_idx}.pdf")

## HELICOID

### Spectrum from data point and literature value

In [ ]:
with open(config.simulated_dataset_dir / "simulation_data.pickle", "rb") as f:
    wavelengths, simulation_data = pickle.load(f)

In [ ]:
A_mcs = SimulationAttenuation(config.mcs_func_path)

In [ ]:
loader = data_loader.DataLoaderHELICOID(
    config.dataset_dir,
    520,
    900
)

mu_a_matrix = loader.absorption_coefs(
    use_diff_oxycco=False,
    use_water_and_fat=True
)

A_predicted = A_mcs.A_concentrations(
    loader.wavelengths,
    mu_a_matrix,
    data_loader.DataLoader.tissue_parameters["gray matter"][0],
    *data_loader.DataLoader.tissue_parameters["gray matter"][1]
)[:, 0]

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=set_size(doc_width, subplots=(1,1)))

ax.plot(
    loader.wavelengths,
    A_predicted,
    label="Literature"
)

# only necessary to set reference pixel
loader.get_attenuation_change("020-01")

gm_spectrum = loader.get_attenuation("020-01")[:, loader.label_map == 1]
gm_spectrum_avg = np.average(gm_spectrum, axis=1)

ax.plot(
    loader.wavelengths,
    loader.get_attenuation("020-01")[:, *loader.reference_pixel],
    label="HELICoiD, reference pixel"
)

ax.plot(
    loader.wavelengths,
    gm_spectrum_avg,
    label="HELICoiD, average"
)

ax.legend()
ax.set_xlabel("Wavelengths (nm)")
ax.set_ylabel("Absorbance")
#ax.set_title("Gray matter absorbance spectra")

fig.savefig(config.eval_dir/"helicoid/data_example.pdf")

### Absolute vs. Diff plot

In [ ]:
with open(config.eval_dir / "helicoid/concentrations_absolute_diff_average.pickle", "rb") as f:
    bar_plot_data = pickle.load(f)
    # (tissue, models, parameters)

In [ ]:
bar_plot_data.shape

In [ ]:
bar_plot_data_normed = np.nan_to_num(bar_plot_data[:, :, :] / bar_plot_data[:, [0], :])

In [ ]:
#models = [
#    "Literature values", "Abs.: Jacques", "Diff.: Jacques", "Abs.: NLLS+MCS", "Diff.: NLLS+MCS", "Abs.: MBLL+MCS", "Diff.: MBLL+MCS"  
#]

models = [
    "Literature values", "Jacques, absolute", "Jacques, difference", "Inverse MC, absolute", "Inverse MC, difference"  
]

In [ ]:
tissues = ["gray matter", "blood vessel"]

In [ ]:
y = np.arange(len(models))
width = 0.125

fig, axs = plt.subplots(2, 1, figsize=set_size(width=doc_width, subplots=(2, 1)), layout='constrained')

for ax, tissue in zip(axs.flat, tissues):
    tissue_idx = ["gray matter",  "tumor", "blood vessel"].index(tissue)
    if tissue == "blood vessel":
        molecule_idxs = [0, 1, 4, 5]
    else:
        molecule_idxs = list(range(6))
    offset_idx = 0
    for molecule_idx, molecule_str in zip(molecule_idxs, np.array(molecules)[molecule_idxs]):
        offset = width * offset_idx
        rects = ax.barh(y + offset, bar_plot_data_normed[tissue_idx, :len(models), molecule_idx], width, label=molecule_str)
        #ax[tissue_idx].bar_label(rects, padding=3)
        offset_idx += 1

    ax.set(xlabel="Concentration fraction to literature values")
    #ax.set_title(f"Reconstructed concentrations, {tissue} tissue")
    ax.set_title(f"{tissue.capitalize()} {'tissue' if tissue != 'blood vessel' else ''}")
    ax.set_yticks(y + width, models)
    ax.invert_yaxis()
    ax.legend(loc="upper right", ncols=3)
    ax.set_xlim(0, 6)

fig.savefig(config.eval_dir/"helicoid/absolute_diff_bar_plot.pdf")


### Bar plot

In [ ]:
with open(config.eval_dir/"helicoid/bar_plot_data.pickle", "rb") as f:
    bar_plot_data = pickle.load(f)
    # tissue, models, params

Remove difference values from MBLL models

In [ ]:
bar_plot_data = bar_plot_data[:, [0, 3, 4, 5, 6], :]

In [ ]:
bar_plot_data_normed = np.nan_to_num(bar_plot_data[:, :, :] / bar_plot_data[:, [0], :])

In [ ]:
models = [
    "Literature values", "Jacques, specific", "Jacques, general", "$\delta$-P1",
    "Inverse MC"
]

In [ ]:
y = np.arange(bar_plot_data_normed.shape[1])
width = 0.125


fig, ax = plt.subplots(2, 1, figsize=set_size(width=doc_width, subplots=(2, 1)), layout='constrained')

for tissue_idx, tissue in enumerate(["gray matter", "tumor"]):
    for molecule_idx, molecule_str in enumerate(molecules):
        offset = width * molecule_idx
        rects = ax[tissue_idx].barh(y + offset, bar_plot_data_normed[tissue_idx, :, molecule_idx], width, label=molecule_str)
        #ax[tissue_idx].bar_label(rects, padding=3)


    ax[tissue_idx].set(xlabel="Concentration percentage to literature values")
    #ax[tissue_idx].set_title(f"Reconstructed concentrations, {tissue} tissue")
    ax[tissue_idx].set_title(f"{tissue.capitalize()} tissue")
    ax[tissue_idx].set_yticks(y + width, models)
    ax[tissue_idx].invert_yaxis()
    ax[tissue_idx].legend(loc="upper right", ncols=3)
    ax[tissue_idx].set_xlim(0, 10)

fig.savefig(config.eval_dir/f"helicoid/bar_plot0.pdf")

In [ ]:
y = np.arange(bar_plot_data_normed.shape[1])
width = 0.125


fig, ax = plt.subplots(1, 1, figsize=set_size(width=doc_width, subplots=(1, 1)), layout='constrained')

bar_offset_idx = 0
for molecule_idx, molecule_str in enumerate(np.array(molecules)):
    if molecule_idx in [2, 3]:
        continue
    offset = width * bar_offset_idx
    rects = ax.barh(y + offset, bar_plot_data_normed[2, :, molecule_idx], width, label=molecule_str)
    #ax[tissue_idx].bar_label(rects, padding=3)
    bar_offset_idx += 1


ax.set(xlabel="Concentration fraction to literature values")
#ax.set_title(f"Reconstructed concentrations, blood vessel tissue")
ax.set_title(f"Blood vessel")
ax.set_yticks(y + width, models)
ax.invert_yaxis()
ax.legend(loc="upper right", ncols=3)
ax.set_xlim(0, 10)

fig.savefig(config.eval_dir/f"helicoid/bar_plot1.pdf")

### Reconstruction times

In [ ]:
patient_ids = ["020-01", "012-01"]

In [ ]:
dfs = []

In [ ]:
for id in patient_ids:
    with open(config.eval_dir/f"helicoid/times_df_{id}.pickle", "rb") as f:
        dfs.append(pickle.load(f))

In [ ]:
concat_df = pandas.concat(dfs, axis=1)
print(concat_df)

In [ ]:
concat_df = concat_df.mean(axis=1)
avg_reconstruction_times = np.empty(4)
avg_reconstruction_times[0] = np.average(concat_df.values[[0, 1, -2, -1]])
avg_reconstruction_times[1] = np.average(concat_df.values[2:4])
avg_reconstruction_times[2:4] = concat_df.values[4:6]

In [ ]:
df2 = pandas.DataFrame(avg_reconstruction_times)
df2.index = ["MBLL", "Jacques", "$\delta$-P1", "Inverse MC"]
df2.columns = ["Time [s]"]

In [ ]:
print(df2)

In [ ]:
df2.to_csv(
    config.eval_dir/"helicoid/times.txt",
    sep="\t",
    index_label="Models",
)

### Images

In [ ]:
patient_id = "020-01"

In [ ]:
loader = data_loader.DataLoaderHELICOID(
    config.dataset_dir,
    400, 1000
)
loader.load_data(patient_id)
print(loader.label_map.shape)

RGB Image from Helicoid, second one with labels and reference pixels.

In [ ]:
config.dataset_dir

#### Helicoid sample image

In [ ]:
bv_pixels = np.nonzero(loader.label_map == 3)
first_bv_pixel = list(zip(*bv_pixels))[0]

In [ ]:
rgb_img = Image.open(config.dataset_dir/f"{patient_id}/image.jpg")
rgb_img.size

In [ ]:
rgb_wavelengths = np.array([708.97, 539.44, 479.06])
rgb_idxs = np.argmin(np.abs(loader.wavelengths[:, None] - rgb_wavelengths[None, :]), axis=0)
img_data_reflectance = loader.reflectance[rgb_idxs, :, :]
img_data_reflectance = img_data_reflectance.transpose(1, 2, 0)

**Note:**
- image has a width of 330, and height of 378 pixels
- Loader (i.e. spectral library) reads image and label map into array of size (height, width)
- imshow also expects image matrix to be in shape (height, width) but prints
- when you take an idx from the np.array, i.e. via list(zip(*np.nonzero(loader.label_map == 1)))[0], then you get (row, width)
- but width is plotted along x-axis, and row along y-axis, therefore you have to switch them when plotting over imshow image with ax.plot()

In [ ]:
label_mask = np.ma.masked_where(loader.label_map.astype(int) == 0, loader.label_map.astype(int))
cmap = ListedColormap(["green", "orange", "red", "white"])

In [ ]:
# comparison between
fig, (ax1, ax2) = plt.subplots(1, 2)
ax1.imshow(rgb_img)
ax2.imshow(img_data_reflectance)
ax1.imshow(label_mask, cmap=cmap)
ax2.imshow(label_mask, cmap=cmap)

In [ ]:
fig = plt.figure(figsize=set_size(doc_width))
grid = AxesGrid(
    fig,
    (1, 1, 1),
    nrows_ncols=(1,2),
    axes_pad=0.1,
)

grid[0].imshow(img_data_reflectance)
grid[1].imshow(img_data_reflectance)
grid[1].imshow(label_mask, cmap=cmap)
grid[0].set_axis_off()
grid[1].set_axis_off()

for tissue_idx, tissue in enumerate(["gray matter", "tumor", "blood vessel"], start=1):
    ref_pixel_row, ref_pixel_col = list(zip(*np.nonzero(loader.label_map == tissue_idx)))[loader.reference_pixel_tissue_ctr]
    grid[1].plot(
        ref_pixel_col,
        ref_pixel_row,
        marker="o",
        markersize="8",
        color="darkblue",

    )

fig.savefig(config.eval_dir/f"helicoid/image_{patient_id}.pdf")

#### Concentration Images

In [ ]:
loader = data_loader.DataLoaderHELICOID(
    config.dataset_dir,
    520, 900
)
loader.load_data(patient_id)
print(loader.label_map.shape)

In [ ]:
with open(config.eval_dir/f"helicoid/reconstructed_concentrations_full_image_{patient_id}.pickle", "rb") as f:
    img_data_flat = pickle.load(f)

In [ ]:
img_data_flat.shape

swap tissue-independent MBLL models with Diffusion and Inverse MC

In [ ]:
img_data_flat2 = img_data_flat.copy()
img_data_flat2[2:4, :, :] = img_data_flat[6:8, :, :]
img_data_flat2[4:8, :, :] = img_data_flat[2:6, :, :]

In [ ]:
np.nan_to_num(img_data_flat2[-1, 5, :]).max()

Reshape and remove scattering params

In [ ]:
img_data = img_data_flat2[:, :6, :].reshape((img_data_flat2.shape[0], 6) + loader.label_map.shape)
img_data.shape # models, params, spectrum

- colorbar für molekül über jeder Spalte
- ylabel = model name für jede Reihe

In [ ]:
models = ["MBLL+MC", "MBLL+$\delta$-P1", r"MBLL,\\gray matter $\bar{L}_p$", r"MBLL,$\bar{L}_p=1$", "Jacques, specific", "Jacques, general", "$\delta$-P1", "Inverse MC"]

In [ ]:
img_size = 2000

In [ ]:
%matplotlib qt

First only the MBLL images, without any modifications.

In [ ]:
img_data_mbll = img_data[:4, :, :, :].copy()

In [ ]:
fig = plt.figure(figsize=set_size(width=img_size/2))
norms = []

grid = AxesGrid(
    fig,
    (1, 1, 1),
    nrows_ncols=(img_data_mbll.shape[:2]),
    axes_pad=0.1,
    cbar_mode="edge",
    cbar_location="top",
    cbar_size="15%",
)

for img_idx in range(np.prod(img_data_mbll.shape[:2])):
    model_idx = img_idx // img_data_mbll.shape[1]
    mol_idx = img_idx % img_data_mbll.shape[1]

    if model_idx == 0:
        norms.append(Normalize(vmin=np.nanmin(img_data_mbll[:, mol_idx, :]), vmax=np.nanmax(img_data_mbll[:, mol_idx, :])))

    img = grid[img_idx].imshow(img_data_mbll[model_idx, mol_idx, :, :], norm=norms[mol_idx], cmap="magma")
    grid[img_idx].set_yticks([])
    grid[img_idx].set_xticks([])

    if mol_idx == 0:
        grid[img_idx].set_ylabel(models[model_idx])

    if model_idx == 0:
        grid.cbar_axes[mol_idx].colorbar(img)
        grid.cbar_axes[mol_idx].set_title(molecules[mol_idx])
        grid.cbar_axes[mol_idx].xaxis.set_ticks_position("top")

fig.savefig(config.eval_dir/f"helicoid/concentration_images_mbll_original_{patient_id}.pdf")

Now only the NLLS images.

In [ ]:
img_data_nlls = img_data[4:, :, :, :].copy()

In [ ]:
fig = plt.figure(figsize=set_size(width=img_size/2))
norms = []

grid = AxesGrid(
    fig,
    (1, 1, 1),
    nrows_ncols=(img_data_nlls.shape[:2]),
    axes_pad=0.1,
    cbar_mode="edge",
    cbar_location="top",
    cbar_size="15%",
)

for img_idx in range(np.prod(img_data_nlls.shape[:2])):
    model_idx = img_idx // img_data_nlls.shape[1]
    mol_idx = img_idx % img_data_nlls.shape[1]

    if model_idx == 0:
        norms.append(Normalize(vmin=np.nanmin(img_data_nlls[:, mol_idx, :, :]), vmax=np.nanmax(img_data_nlls[:, mol_idx, :, :])))

    img = grid[img_idx].imshow(img_data_nlls[model_idx, mol_idx, :, :], norm=norms[mol_idx], cmap="magma")
    grid[img_idx].set_yticks([])
    grid[img_idx].set_xticks([])

    if mol_idx == 0:
        grid[img_idx].set_ylabel(models[4 + model_idx])

    if model_idx == 0:
        grid.cbar_axes[mol_idx].colorbar(img)
        grid.cbar_axes[mol_idx].set_title(molecules[mol_idx])
        grid.cbar_axes[mol_idx].xaxis.set_ticks_position("top")

fig.savefig(config.eval_dir/f"helicoid/concentration_images_nlls_{patient_id}.pdf")

Now every model. To make MBLL models comparable with absolute values of NLLS models, we add literature values (per tissue) to MBLL image data.

In [ ]:
for tissue_idx, tissue in enumerate(["unlabeled", "gray matter", "tumor", "blood vessel"]):
    tissue_key = tissue if tissue != "unlabeled" else "gray matter"
    img_data[:2, :, (loader.label_map == tissue_idx)] += data_loader.DataLoader.tissue_parameters[tissue_key][0][None, :, None]

img_data[2:4, :, (loader.label_map != 4)] += data_loader.DataLoader.tissue_parameters["gray matter"][0][None, :, None]

This code does steps above, but removes MBLL const, for better visualization:

In [ ]:
remove_mbll = False
if remove_mbll:
    ### without MBLL const PL
    img_data_flat2 = np.empty((img_data_flat.shape[0]-1, img_data_flat.shape[1], img_data_flat.shape[2]))
    img_data_flat2[:2, :, :] = img_data_flat[:2, :, :]
    img_data_flat2[2, :, :] = img_data_flat[6, :, :]
    img_data_flat2[3:7, :, :] = img_data_flat[2:6, :, :] 
    img_data = img_data_flat2[:, :6, :].reshape((img_data_flat2.shape[0], 6) + loader.label_map.shape)
    img_data.shape # models, params, spectrum
    for tissue_idx, tissue in enumerate(["unlabeled", "gray matter", "tumor", "blood vessel"]):
        tissue_key = tissue if tissue != "unlabeled" else "gray matter"
        img_data[:2, :, (loader.label_map == tissue_idx)] += data_loader.DataLoader.tissue_parameters[tissue_key][0][None, :, None]

    img_data[2, :, (loader.label_map != 4)] += data_loader.DataLoader.tissue_parameters["gray matter"][0][None, :]
    models = ["MBLL+MC", "MBLL+$\delta$-P1", r"MBLL, gray matter $\bar{L}_p$", "Jacques, specific", "Jacques, general", "Diffusion", "NLLS+MC"]
else:
    print("Skipped.")

In [ ]:
fig = plt.figure(figsize=(set_size(img_size)))
#norm = Normalize(vmin=np.min(img_data), vmax=np.max(img_data))
norms = []

grid = AxesGrid(
    fig,
    (1, 1, 1),
    nrows_ncols=(img_data.shape[:2]),
    axes_pad=0.1,
    cbar_mode="edge",
    cbar_location="top",
    cbar_size="15%",
)

for img_idx in range(np.prod(img_data.shape[:2])):
    model_idx = img_idx // img_data.shape[1]
    mol_idx = img_idx % img_data.shape[1]

    if model_idx == 0:
        norms.append(Normalize(vmin=np.nanmin(img_data[:, mol_idx, :, :]), vmax=np.nanmax(img_data[:, mol_idx, :, :])))

    img = grid[img_idx].imshow(img_data[model_idx, mol_idx, :, :], norm=norms[mol_idx], cmap="magma")
    grid[img_idx].set_yticks([])
    grid[img_idx].set_xticks([])

    if mol_idx == 0:
        grid[img_idx].set_ylabel(models[model_idx])

    if model_idx == 0:
        grid.cbar_axes[mol_idx].colorbar(img)
        grid.cbar_axes[mol_idx].set_title(molecules[mol_idx])
        grid.cbar_axes[mol_idx].xaxis.set_ticks_position("top")
        

fig.savefig(config.eval_dir/f"helicoid/concentration_images_shared_cbar_{patient_id}.pdf")

**Only for display on PC, for analysis.** Same as above, but transposed for larger screen.

In [ ]:
fig = plt.figure(figsize=(100, 50))

norms = []

grid = AxesGrid(
    fig,
    (1, 1, 1),
    nrows_ncols=(img_data.shape[1], img_data.shape[0]),
    axes_pad=0.1,
    cbar_mode="edge",
    cbar_location="right",
    cbar_size="15%",
)

for img_idx in range(np.prod(img_data.shape[:2])):
    model_idx = img_idx % img_data.shape[0]
    mol_idx = img_idx // img_data.shape[0]

    if model_idx == 0:
        norms.append(Normalize(vmin=np.nanmin(img_data[:, mol_idx, :, :]), vmax=np.nanmax(img_data[:, mol_idx, :, :])))
    
    img = grid[img_idx].imshow(img_data[model_idx, mol_idx, :, :], norm=norms[mol_idx], cmap="magma")
    grid[img_idx].set_yticks([])
    grid[img_idx].set_xticks([])

    if model_idx == 0:
        grid[img_idx].set_ylabel(molecules[mol_idx])
        grid.cbar_axes[mol_idx].colorbar(img)

    if mol_idx == 0:
        grid[img_idx].set_title(models[model_idx])

Now MBLL and NLLS data together, but using two colormaps. We use MBLL data with added literature values, to report absolute concentrations for both tissues.

In [ ]:
fig = plt.figure(figsize=(17, 17))

num_mbll_models = 4

for i, cur_img_data in enumerate(np.split(img_data, np.array([num_mbll_models]), axis=0)):
    norms = []
    num_models, num_mols = cur_img_data.shape[:2]
    grid = AxesGrid(
        fig,
        (2, 1, i+1),
        nrows_ncols=(num_models, num_mols),
        axes_pad=0.1,
        cbar_mode="edge",
        cbar_location="top",
        cbar_size="15%"
    )

    for img_idx in range(num_models*num_mols):
        model_idx = img_idx // num_mols
        mol_idx = img_idx % num_mols

        if model_idx == 0:
            norms.append(Normalize(vmin=np.nanmin(cur_img_data[:, mol_idx, :, :]), vmax=np.nanmax(cur_img_data[:, mol_idx, :, :])))
        
        img = grid[img_idx].imshow(cur_img_data[model_idx, mol_idx, :, :], norm=norms[mol_idx], cmap="magma")
        grid[img_idx].set_yticks([])
        grid[img_idx].set_xticks([])

        if model_idx == 0:
            grid.cbar_axes[mol_idx].colorbar(img)
            grid.cbar_axes[mol_idx].set_title(molecules[mol_idx])
            grid.cbar_axes[mol_idx].xaxis.set_ticks_position("top")
        
        if mol_idx == 0:
            grid[img_idx].set_ylabel(models[num_mbll_models*i + model_idx])

fig.savefig(config.eval_dir/f"helicoid/concentration_images_seperate_cbar_{patient_id}.pdf")

Same as above but two different figures. With molecule units and fraction in percent (i.e. times 100).

In [ ]:
img_data_tmp = img_data.copy()
img_data_tmp[:, -2:, :, :] *= 100
num_mbll_models = 4
molecules_units_imgs = [r"$c_\text{Hb02}$ [mM]", r"$c_\text{Hbb}$ [mM]", r"$c_\text{oxCCO}$ [mM]", r"$c_\text{redCCO}$ [mM]", r"$f_\text{H20}$ [\%]", r"$f_\text{Fat}$ [\%]"]

for i, cur_img_data in enumerate(np.split(img_data_tmp, np.array([num_mbll_models]), axis=0)):
    fig = plt.figure(figsize=(9.4,9.4))
    norms = []
    num_models, num_mols = cur_img_data.shape[:2]
    grid = AxesGrid(
        fig,
        (1, 1, 1),
        nrows_ncols=(num_models, num_mols),
        axes_pad=0.1,
        cbar_mode="edge",
        cbar_location="top",
        cbar_size="15%"
    )

    for img_idx in range(num_models*num_mols):
        model_idx = img_idx // num_mols
        mol_idx = img_idx % num_mols

        if model_idx == 0:
            norms.append(Normalize(vmin=np.nanmin(cur_img_data[:, mol_idx, :, :]), vmax=np.nanmax(cur_img_data[:, mol_idx, :, :])))
        
        img = grid[img_idx].imshow(cur_img_data[model_idx, mol_idx, :, :], norm=norms[mol_idx], cmap="magma")
        grid[img_idx].set_yticks([])
        grid[img_idx].set_xticks([])

        if model_idx == 0:
            grid.cbar_axes[mol_idx].colorbar(img)
            grid.cbar_axes[mol_idx].set_title(molecules_units_imgs[mol_idx])
            grid.cbar_axes[mol_idx].xaxis.set_ticks_position("top")
        
        if mol_idx == 0:
            grid[img_idx].set_ylabel(models[num_mbll_models*i + model_idx])

    fig.savefig(config.eval_dir/f"helicoid/concentration_images_seperate_cbar_{patient_id}_{i}.pdf")

### Prediction error

In [ ]:
with open(config.eval_dir/"helicoid/prediction_error_df.pickle", "rb") as f:
    df = pickle.load(f)

In [ ]:
df

In [ ]:
df2 = pandas.DataFrame(df.values[-4:, :])
df2.index = ["Jacques, specific", "Jacques, general", "$\delta$-P1", "Inverse MC"]
df2.columns = ["Gray matter", "Tumor", "Blood vessel", "Average"]

In [ ]:
df2.to_csv(
    config.eval_dir/"helicoid/prediction_error.txt",
    sep="\t",
    index_label="Models"
)